<h1>
Problema das Rainhas
</h1>

Em um tabuleiro 𝑛×𝑛, uma rainha é colocada em um quadrado, irá dominar todos os quadrados que estiverem na mesma linha, coluna e diagonais. A ideia por trás deste probelma é achar a quantidade mínima de rainhas necessárias para dominar o tabuleiro inteiro. Dominar, neste problema, significa cobrir todos os quadrados possíveis sendo atacados por rainhas incluindo aqueles onde as rainhas se encontram.

Primeiro, vamos começar definindo o tabuleiro como é feito no artigo. Vamos começar por um tabuleiro 4x4, igual começa no artigo:

In [1]:
import numpy as np
CHESSBOARD_DIMENSION = 8
N = CHESSBOARD_DIMENSION
chessBoardDemo = np.array([x+1 for x in range(N*N)])

chessBoardDemo = chessBoardDemo.reshape(N,N)
print(chessBoardDemo)

[[ 1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16]
 [17 18 19 20 21 22 23 24]
 [25 26 27 28 29 30 31 32]
 [33 34 35 36 37 38 39 40]
 [41 42 43 44 45 46 47 48]
 [49 50 51 52 53 54 55 56]
 [57 58 59 60 61 62 63 64]]


Agora, criar a função para verificar quadrados do tabuleiro que estão sendo dominados por todas as rainhas e retornar os números que estão sendo dominados:

In [2]:
def dominatedSet(queens, chessBoard, verbose=False):
    S = set()

    N = chessBoard.shape[0]

    if verbose:
        print('tabuleiro que chega no dominatedSet')
        print(chessBoard)
    
    for queen in queens:
        queenBox = queen
        queenPosition = np.where(chessBoard == queenBox)
        i = queenPosition[0][0]
        j = queenPosition[1][0]
        if verbose: 
            print("posição matricial: (", i,j, ")" , 'valor: ', queenBox)

        if(verbose):
            # quadrados dominados na horizontal:
            print(chessBoard[i,:])
            # quadrados dominados na vertical:
            print(chessBoard[:,j])
       
        S.update(chessBoard[i,:])
        S.update(chessBoard[:,j])
        for d in range(-N+1,N):
             # diagonal
            diag = chessBoard.diagonal(d)
            # diagonal invertida
            invertDiag = np.fliplr(chessBoard).diagonal(d)
            if queenBox in diag:
                if verbose:
                    print(diag)
                S.update(diag)
            if queenBox in invertDiag:
                if verbose:
                    print(invertDiag)
                S.update(invertDiag)
    return S
        

O código abaixo vai então gerar a área de dominância para uma única rainha na posição (3,3) do tabuleiro, ou também podemos dizer, no valor 28 da matriz. 

<b>Para ficar mais claro, o número do quadrado em que a rainha se encontra nós vamos chamar de posição numérica. Então o quadrado 28, em que essa rainha se encontra, é sua posição numérica. A dupla (3,3) e todas as outras nós vamos chamar de posição matricial </b>

In [3]:
S = dominatedSet([28], chessBoardDemo, verbose=False)
print(S)

{1, 4, 7, 10, 12, 14, 19, 20, 21, 25, 26, 27, 28, 29, 30, 31, 32, 35, 36, 37, 42, 44, 46, 49, 52, 55, 60, 64}


A função que mede o quão próximo estamos de uma boa solução no artigo é dada por:

$ f(x) = {|S| \div |G|}  $

Nesta função, se o resultado for menor que 1, temos que existem alguns ou pelo menos 1 quadrado que não está numa área de dominância das rainhas. Se for 1, significa que as rainhas dominaram todos os quadrados do tabuleiro. 

No código, calcularemos essa função assim:

In [4]:
len(S)/ (N*N) 

0.4375

<h2>Codificação dos indivíduos </h2>

O conjunto de indivíduos será representado por uma matriz onde cada linha é um solução candidata e as colunas são posições das rainhas no tabuleiro para aquela solução. As posições das rainhas serão dadas pelos valores das posições numéricas definidas como um binário de 8 dígitos.
Seria Assim:


In [5]:
rainha1 = format(23,'08b')
rainha2 = format(43,'08b')
rainha3 = format(12,'08b')
rainha4 = format(10,'08b')

np.matrix([[rainha1, rainha2], [rainha3, rainha4]])

matrix([['00010111', '00101011'],
        ['00001100', '00001010']], dtype='<U8')

No caso, a primeira solução candidata tem as rainhas nas posições numéricas 23 e 43, e segunda solução candidata tem rainhas nas posições 12 e 10

<h2>Geração inicial de indivíduos </h2>

A primeira geração consiste de 100 indivíduos (soluções candidatas) geradas aleatoriamente, mantendo a certeza de que em cada possível solução as rainhas estão em posições diferentes. Vamos fazer uma função que faz isso. A função vai receber como parâmetro o número de rainhas que queremos para as soluções possíveis e a dimensão do tabuleiro.

In [6]:
from random import seed
from random import randint

seed(3)

value = randint(0, N*N)
value

def gen_individuals(N_queens, N, N_individuals, verbose=True): 
    print('')
    A = np.empty((1,N_queens), dtype='str')
    for i in range (0,N_individuals):
        newRow = []
        for j in range (0,N_queens):
            randVal = randint(1,N*N)
            randInBin = format(randVal,'08b')
            while randInBin in newRow: 
                randVal = randint(1,N*N)
                randInBin = format(randVal,'08b')
            newRow.append(randInBin)
        A = np.vstack([A, newRow])
    # apaga a primeira linha que é gerada na inicialização da matriz
    A = np.delete(A,0, 0)
    return A



Vendo abaixo uma amostra de dois indivíduos. Cada linha é uma solução possível e essas soluções possuem duas rainhas. A função também permite que a gente decida quantos indivíduos queremos gerar para a matriz A (número de linhas) assim como quantas rainhas pode ter cada indivíduo (número de colunas)

In [7]:
A = gen_individuals(2,N,100)
print(A[0])
print(A[99])


['00010001' '00110000']
['00011111' '00000011']


Já vou deixar pronta uma função que retorna o fitness de cada indivíduo:

In [8]:
def darwinize_individual(ind, chessBoard):
    indInInteger = []
    N = chessBoard.shape[0]
    for queen in ind:
        inIntQueen = int(queen,2)
        indInInteger.append(inIntQueen)
    S = dominatedSet(indInInteger, chessBoard, verbose=False)
    ## o jeito de calcular o fitness como foi dito anteriormente
    return len(S)/ (N*N) 
    

darwinize_individual(A[0], chessBoardDemo)

# uma coisa que dá pra ver é que os individuos passam todos da metade de dominância do tabuleiro, pelo menos com duas rainhas
for individual in A:
    fit = darwinize_individual(individual, chessBoardDemo)
    if fit < 0.5:
        print(individual)


Vou deixar aqui também uma função que gera os fitness values da população inteira como forma de diminuir repetição de código

In [9]:
def getFitness(population, chessBoard, verbose=False):
    fitness_values = np.empty((1), dtype='int64')
    for indi in population:
        if verbose: print('individo no getFitness', indi)
        fit = darwinize_individual(indi, chessBoard)
        fitness_values = np.vstack([fitness_values, fit])
    fitness_values = np.delete(fitness_values,0, 0)
    return fitness_values

A função abaixo devolve os 50% mais adaptados, ou seja, os 50% que tiveram maior fitness.

In [10]:
def moreAdapted(population, chessBoard):
    n_queens = population.shape[1]
    fitness_values = getFitness(population, chessBoard)

    # Selecionará os 50% individuos com maior fitness_values
    n_ind = len(population)
    individuals = np.empty((1,n_queens), dtype='str')
    fit = np.empty((1), dtype='str')

    for i in range(n_ind):
        max_fitness_idx = np.argmax(fitness_values)
        fit = np.vstack([fit,fitness_values[max_fitness_idx]])
        individuals = np.vstack([individuals, population[max_fitness_idx]])
        fitness_values[max_fitness_idx] = -99999999999
    individuals = np.delete(individuals,0, 0)
    fit = np.delete(fit,0,0)
    individuals = individuals[0:int(n_ind/2)]
    fit = fit[0:int(n_ind/2)]
    return individuals

Função da Roullete wheel para selecionar 2 indivíduos para crossover:

In [11]:
def roulette_wheel(population, chessBoard):
    fitness_values = getFitness(population, chessBoard)

    y = fitness_values.astype(np.float)
    Soma = np.sum(y)
    Prob_escolhido = np.empty((1), dtype='float')
  
    for i in y:   
        Prob_escolhido = np.vstack([Prob_escolhido, i/Soma])
    Prob_escolhido = np.delete(Prob_escolhido,0,0) 
    Prob_escolhido = np.transpose(Prob_escolhido) 
    indices = np.random.choice(len(population),size= 2 ,p=Prob_escolhido.flatten())
    ind = population[indices]
    return ind

In [12]:
B = moreAdapted(A, chessBoardDemo)

O crossover e a mutação não podem gerar valores que estão fora da dimensão. para isso o artigo descreve um método que consiste em mudar os valores dos dígitos 0 e 1 para que o valor encaixe no tamanho do tabuleiro. A ideia que pensamos é diferente: nós vamos "ciclar" o valor usando o operador de módulo e nisso vai cair num valor aceito.

Eis a função para a validação do crossover e da mutação:

In [13]:
def validate_dna(dna, verbose=False):
    # N é variaǘel global, tome cuidado
    if verbose: 
        print('valida o dna')
        print('dimensão', N)
    dnaInInt = int(dna,2)
    
    if verbose: print('valor anterior: ', dnaInInt)
    if dnaInInt > N*N:
        dnaInInt = dnaInInt % (N*N)
        if dnaInInt == 0: dnaInInt +=1
        dna = format(dnaInInt,'08b')
        if verbose: print('valor após validação: ', dnaInInt)
        return dna
    elif dnaInInt == 0: 
        dnaInInt += 1
        dna = format(dnaInInt,'08b')
        return dna
    else: return dna    




Função para fazer o crossover:

In [14]:
def crossover(parents, verbose=False):
    DNA_SIZE = 8
    individual1 = parents[0]
    individual2 =  parents[1]
    n_queens = individual1.shape[0]
    crossedIndividuals = np.empty((1,n_queens), dtype='str')
    if verbose: print('pais', parents)
    DNA1 = []
    DNA2 = []
    for i in range(0, n_queens):
        pos = randint(1, DNA_SIZE-1)
        if verbose: print('posição de troca: ', pos)
        dna1=individual1[i]
        dna2=individual2[i]
        generated1, generated2 = (dna1[:pos]+dna2[pos:], dna2[:pos]+dna1[pos:])
        generated1 = validate_dna(generated1)
        DNA1.append(generated1)
        generated2 = validate_dna(generated2)
        DNA2.append(generated2)
    crossedIndividuals = np.vstack([crossedIndividuals, [DNA1, DNA2]])
    crossedIndividuals = np.delete(crossedIndividuals,0, 0)
    return crossedIndividuals

Abaixo um exemplo do funcionamento da função de crossover. O ponto de crossover é diferente para cada cruzamento de rainhas.

In [15]:
parents = roulette_wheel(B, chessBoardDemo)
children = crossover(parents, verbose=True)
print('filhos:' ,children)

pais [['00100011' '00011111']
 ['00110101' '00100110']]
posição de troca:  5
posição de troca:  5
filhos: [['00100101' '00011110']
 ['00110011' '00100111']]


Função de mutação:

In [16]:
def mutate(individual, verbose=False, debug=False):
    DNA_SIZE = 8
    top = 100
    if debug: top = 1
    if(randint(1,top) <= 5):
        mutatedIndividual =[]
        for DNA in individual:
                if verbose: print('antes da mutação', DNA)
                DNA = list(DNA)
                # pode acontecer de mudar a mesma posição duas vezes. Acho que não tem problema
                pos1 = randint(0, DNA_SIZE-1)
                pos2 =  randint(0, DNA_SIZE-1)
                if verbose: print('mudou as posições:', pos1, pos2)
                if DNA[pos1] == "1": DNA[pos1] = "0"
                elif DNA[pos1] == "0": DNA[pos1] = "1"
                if DNA[pos2] == "1": DNA[pos2] = "0"
                elif DNA[pos2] == "0": DNA[pos2] = "1" 
                DNAinString = "".join(DNA)
                DNA = validate_dna(DNAinString)
                mutatedIndividual.append(DNA)
        if verbose: print('após a mutação', mutatedIndividual)
        return mutatedIndividual 
    else:
        return individual            


In [17]:
mutated = mutate(children[0],verbose=True)
validate_dna(mutated[0])

'00100101'

Agora, já temos a função que gera a população inicial, A função que gera os primeiros 100 indivíduos, a função que filtra os mais adaptados, a função de roleta, de crossover e de mutação.
Pelo que está descrito no artigo, o processo consiste em gerar os primeiros 100 indivíduos, em seguida filtrar os melhores 50%, 
depois escolher os indivíduos para crossover por meio da roleta, depois cruzar esses indivíduos, realizar a mutação e repetir esse processo mil vezes. 

Além disso, devem ser feitas alterações nos indivíduos caso eles ultrapassem as dimensões do tabuleiro após croosover e mutação.

In [18]:
def do_iterations(n_iters, n_queens, chessBoard_dimension):
    # constrói o tabuleiro
    N = chessBoard_dimension
    chessBoard = np.array([x+1 for x in range(N*N)])
    chessBoard = chessBoard.reshape(N,N)

    # gera os primeiros 100 indivíduos:
    A = gen_individuals(n_queens,N,100)
    i = 1
    while i <= n_iters:
        # já filtra o top 50%
        A = moreAdapted(A, chessBoard)  
        
        # vamos gerar mais 50 indivíduos à partir de 25 pares selecionados de pares com a roleta, e esses pares vão 
        # gerar dois indivíduos cada um. No final temos 100 indivídus.
        parentsSelected = []
        for j in range (0,25):
            parents = roulette_wheel(A, chessBoard)
            parentsSelected.append(parents)
        
        # agora fazemos os pais cruzarem. Cada pai vai gerar um par de filhos.
        childrenCreated = []
        for parents in parentsSelected:
            children = crossover(parents)
            childrenCreated.append(children)

        # aplica a mutação nos filhos, com base na chance de 5% definida na função
        for idx, children in enumerate(childrenCreated):
            childrenCreated[idx][0] = mutate(children[0])
            childrenCreated[idx][1] = mutate(children[1])

        # separar os pares e alocá-los numa matriz:
        AllIndividuals = np.empty((1,n_queens), dtype='str')
        for k in range(0, len(childrenCreated)):
            AllIndividuals = np.vstack(([ AllIndividuals, childrenCreated[k][0]]))
            AllIndividuals = np.vstack(([ AllIndividuals, childrenCreated[k][1]]))
            AllIndividuals = np.vstack(([ AllIndividuals, parentsSelected[k][0]]))
            AllIndividuals = np.vstack(([ AllIndividuals, parentsSelected[k][1]]))
        AllIndividuals = np.delete(AllIndividuals,0, 0)


        fitness_values = getFitness(AllIndividuals, chessBoard)
        A = AllIndividuals
        
        print('melhor indivíduo da iteração', i,' :' , fitness_values[np.argmax(fitness_values)])
        print('posição das rainhas do melhor individuo',A[np.argmax(fitness_values)])
        i += 1
    return 1


In [19]:
N = 9
do_iterations(1000,5,N)


melhor indivíduo da iteração 1  : [0.95061728]
posição das rainhas do melhor individuo ['00111101' '00010011' '01000111' '00110010' '00000110']
melhor indivíduo da iteração 2  : [0.92592593]
posição das rainhas do melhor individuo ['01000001' '00001100' '00011011' '00100010' '01001101']
melhor indivíduo da iteração 3  : [0.9382716]
posição das rainhas do melhor individuo ['00000111' '00010010' '00100110' '00111110' '00110001']
melhor indivíduo da iteração 4  : [0.95061728]
posição das rainhas do melhor individuo ['00110101' '00011110' '00000001' '01001011' '00100000']
melhor indivíduo da iteração 5  : [0.9382716]
posição das rainhas do melhor individuo ['00000111' '00010010' '00100110' '00111110' '00110001']
melhor indivíduo da iteração 6  : [0.9382716]
posição das rainhas do melhor individuo ['00000111' '00010010' '00100110' '00111110' '00110001']
melhor indivíduo da iteração 7  : [0.95061728]
posição das rainhas do melhor individuo ['00110101' '00011111' '00001010' '01001001' '00100

melhor indivíduo da iteração 58  : [0.97530864]
posição das rainhas do melhor individuo ['00000001' '00001100' '00101011' '01001100' '00110001']
melhor indivíduo da iteração 59  : [0.97530864]
posição das rainhas do melhor individuo ['00000001' '00001100' '00101011' '01001100' '00110001']
melhor indivíduo da iteração 60  : [0.97530864]
posição das rainhas do melhor individuo ['00000001' '00001100' '00101011' '01001100' '00110001']
melhor indivíduo da iteração 61  : [0.97530864]
posição das rainhas do melhor individuo ['00000001' '00001100' '00101011' '01001100' '00110001']
melhor indivíduo da iteração 62  : [0.97530864]
posição das rainhas do melhor individuo ['00000001' '00001100' '00101011' '01001100' '00110001']
melhor indivíduo da iteração 63  : [0.97530864]
posição das rainhas do melhor individuo ['00000001' '00001100' '00101011' '01001100' '00110001']
melhor indivíduo da iteração 64  : [0.97530864]
posição das rainhas do melhor individuo ['00000001' '00001100' '00101011' '0100110

KeyboardInterrupt: 